In [1]:
import tensorflow as tf
import numpy as np 
import os
import cv2
import pickle
import imageio
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Concatenate, Conv2DTranspose, Conv2D, MaxPooling2D, Activation, Dropout, UpSampling2D
from keras.models import Model, load_model, Sequential
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [2]:
IMG_WIDTH = 320
IMG_HEIGHT = 176
IMG_CHANNELS = 3

In [ ]:
def create_model(input_shape):
    input = tf.keras.layers.Input(input_shape)
    s = tf.keras.layers.Lambda(lambda x: x / 255)(input)
    
    c1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(s)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)
    
    c2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)
    
    c3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.1)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)
    
    c4 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.1)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)
    
    c5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c5)
    
    u6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
    u6 = tf.keras.layers.Concatenate()([u6,c4])
    c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u6)
    c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c6)
    
    u7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
    u7 = tf.keras.layers.Concatenate()([u7,c3])
    c7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c7)
    
    u8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c7)
    u8 = tf.keras.layers.Concatenate()([u8,c2])
    c8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c8)
    
    u9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(c8)
    u9 = tf.keras.layers.Concatenate(axis=3)([u9,c1])
    c9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal',padding='same')(c9)
    
    output = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)
    model = tf.keras.Model(inputs=[input], outputs=[output])
    return model

In [3]:
filenames=os.listdir("./data/")        #lấy tất cả ảnh từ folder data
labelf=os.listdir("./label/")          #lấy tất cả ảnh từ folder label
filenames.sort()
labelf.sort()

labels=[]
images=[]
#đọc tất cả ảnh
for i in filenames:
    img=cv2.imread("data/{}".format(i))
    img = cv2.resize(img, (320, 176))
    images.append(img)
#đọc tất cả label
for i in labelf:
    img = cv2.imread("label/{}".format(i))
    img = cv2.resize(img, (320, 176))
    img=img[:,:,0]
    img=img.reshape(176, 320,1)
    labels.append(img)

In [4]:
print(np.array(labels).shape)
print(np.array(images).shape)

(1871, 176, 320, 1)
(1871, 176, 320, 3)


In [5]:
images = np.array(images)
labels = np.array(labels)
labels = labels / 255          #đưa các giá trị output từ 0-255 về 0-1        
images, labels = shuffle(images, labels)
#chia tập dữ liệu thành 80% cho training, và 20% cho validation 
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2)

In [ ]:
model = create_model((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
datagen = ImageDataGenerator(channel_shift_range=0.2, brightness_range=[0.7,1.3], rotation_range=40, rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)  

In [ ]:
earlystop = EarlyStopping(patience=10)   
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',   #điều cần quan tâm là độ chính xác nếu độ chính xác 
                                                                 #không tăng thì lr sẽ giảm
                                            patience=3,          #số lượng epoch mà sau đó không cải thiện chương trình sẽ stop
                                            verbose=1,           #có cập nhật và hiển thị trạng thái khi reduce
                                            factor=0.5,          #new_lr=factor*lr
                                            min_lr=1e-6)      #giới hạn lr
checkpoint = ModelCheckpoint('models2/model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')
callb=[learning_rate_reduction,checkpoint]

# results = model.fit(X_train,y_train, steps_per_epoch=len(X_train)/16, epochs=100, verbose=1,batch_size=16, validation_data=(X_val, y_val), callbacks=callb)
model.fit_generator(datagen.flow(X_train, y_train, batch_size=16), steps_per_epoch=len(X_train)/16,
    epochs=200, verbose=1, validation_data=(X_val, y_val), callbacks=callb)

In [7]:
def create_model1(input_shape):
    # Create the actual neural network here
    input_img = Input(input_shape, name='img')

    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (input_img)
    c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
    c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
    c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)

    u5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c4)
    u5 = Concatenate()([u5, c3])
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same') (u5)
    c6 = Conv2D(32, (3, 3), activation='relu', padding='same') (c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = Concatenate()([u7, c2])
    c7 = Conv2D(16, (3, 3), activation='relu', padding='same') (u7)
    c7 = Conv2D(16, (3, 3), activation='relu', padding='same') (c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = Concatenate()([u8, c1])
    c8 = Conv2D(8, (3, 3), activation='relu', padding='same') (u8)
    c8 = Conv2D(8, (3, 3), activation='relu', padding='same') (c8)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c8)

    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [8]:
model1 = create_model1((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img (InputLayer)            [(None, 176, 320, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 176, 320, 8)          224       ['img[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)           (None, 176, 320, 8)          584       ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 88, 160, 8)           0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [9]:
datagen = ImageDataGenerator(channel_shift_range=0.2, brightness_range=[0.7,1.3])  

In [10]:
earlystop = EarlyStopping(patience=10)   
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',   #điều cần quan tâm là độ chính xác nếu độ chính xác 
                                                                 #không tăng thì lr sẽ giảm
                                            patience=3,          #số lượng epoch mà sau đó không cải thiện chương trình sẽ stop
                                            verbose=1,           #có cập nhật và hiển thị trạng thái khi reduce
                                            factor=0.5,          #new_lr=factor*lr
                                            min_lr=1e-6)      #giới hạn lr
checkpoint = ModelCheckpoint('models3/model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')
callb=[learning_rate_reduction,checkpoint]

# results = model.fit(X_train,y_train, steps_per_epoch=len(X_train)/16, epochs=100, verbose=1,batch_size=16, validation_data=(X_val, y_val), callbacks=callb)
model1.fit_generator(datagen.flow(X_train, y_train, batch_size=16), steps_per_epoch=len(X_train)/16,
    epochs=500, verbose=1, validation_data=(X_val, y_val), callbacks=callb)

Epoch 1/500


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22668\311015497.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model1.fit_generator(datagen.flow(X_train, y_train, batch_size=16), steps_per_epoch=len(X_train)/16,


22/93 [======>.......................] - ETA: 1:21 - loss: 13.8990 - accuracy: 0.4992

KeyboardInterrupt: 